Install the required libraries

In [ ]:
!pip install langchain langchain-community langchain-together sentence-transformers accelerate faiss-cpu PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_together import Together
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
import os
from google.colab import files
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter


Upload the IPC pdf

In [ ]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
reader = PdfReader(pdf_path)
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"


Saving ipc1.pdf to ipc1.pdf


Split Text into Chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(text)

Convert Text to Embeddings & Create FAISS Vector Store

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs={"trust_remote_code": True, "revision": "REVISION_KEY"}
)

db = FAISS.from_texts(chunks, embeddings)
db.save_local("ipc_vector_db")

db_retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})


<ipython-input-5-0832023d4dfe>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

Define the Prompt Template

In [ ]:
prompt_template = """<s>[INST]This is a chat template. As a legal chatbot specializing in Indian Penal Code queries, """
"""your primary objective is to provide accurate and concise information based on the user's questions. """
"""Do not generate your own questions and answers. Adhere strictly to the instructions provided, """
"""offering relevant context from the knowledge base while avoiding unnecessary details. """
"""Your responses will be brief, professional, and contextually relevant. """
"""CONTEXT: {context}\nCHAT HISTORY: {chat_history}\nQUESTION: {question}\nANSWER: </s>[INST] """

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question', 'chat_history']
)

Set Up the LLM

In [ ]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question', 'chat_history']
)

# Set up TogetherAI API
os.environ["TOGETHER_AI"] = "MY_API_KEY"
TOGETHER_AI_API = os.environ["TOGETHER_AI"]
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.5,
    max_tokens=1024,
    together_api_key=TOGETHER_AI_API
)

 Set Up Conversational Retrieval Chain

In [ ]:
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history", return_messages=True)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    memory=memory,
    retriever=db_retriever,
    return_source_documents=False,
    combine_docs_chain_kwargs={'document_variable_name': 'context'},
    output_key="answer"
)

<ipython-input-8-1ad3f3f377a3>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history", return_messages=True)


Function to Interact with the Chatbot

In [ ]:
def chatbot():
    print("IPC Chatbot: Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        response = get_response(user_input)
        print("Chatbot:", response)

def get_response(user_input):
    result = qa.invoke({"question": user_input, "chat_history": []})
    return result["answer"]

if __name__ == "__main__":
    chatbot()


IPC Chatbot: Type 'exit' to end the conversation.
You: What is rape?
Chatbot:  Rape is an act of sexual intercourse committed by a man with a woman without her consent. It can occur in various situations as defined in the given context, such as:
1. When such an assault as may reasonably cause the apprehension that death will otherwise be the consequence of such assault.
2. When such an assault as may reasonably cause the apprehension that grievous hurt will otherwise be the consequence of such assault.
3. With the intention of committing rape.
4. With the intention of gratifying unnatural lust.
5. With the intention of kidnapping or abducting.
6. If the perpetrator is a relative, guardian, teacher, or person in a position of trust or authority towards the woman.
7. During communal or sectarian violence.
8. On a woman knowing her to be pregnant.
9. On a woman incapable of giving consent.
10. Being in a position of control or dominance over a woman and committing rape on such woman.
11. 